In [0]:
#all spark imports
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
#instantiate the spark session
spark = SparkSession.builder.appName("Demo").getOrCreate()

In [0]:
# set the shuffle partition same as number of cpu cores to improve performance 
# spark.conf.set("spark.sql.shuffle.partitions", 2)

# Typical Application Flow

### Source -> Load -> Process -> Results -> Destination

## RDD

### Resilient, Partitioned, Distributed, Immutable collection of data.

In [0]:
feelistRDD = spark.sparkContext.textFile("/FileStore/tables/feelist.txt", 4)

In [0]:
# get number of partitions
feelistRDD.getNumPartitions()

Out[6]: 4

### How Spark run code in parallel?

In [0]:
feelistRDD.count()

Out[7]: 24

### Transformations

In [0]:
feelistRDD.collect()

Out[8]: ['1, Mech, 10000',
 '6, CS, 20000',
 '7, Elec, 5000',
 '2, Mech, 12000',
 '4, Mech, 16000',
 '3, Elec, 18000',
 '9, CS, 16000',
 '5, Mech, 6000',
 '1, Mech, 10000',
 '6, CS, 20000',
 '7, Elec, 5000',
 '2, Mech, 12000',
 '4, Mech, 16000',
 '3, Elec, 18000',
 '9, CS, 16000',
 '5, Mech, 6000',
 '1, Mech, 10000',
 '6, CS, 20000',
 '7, Elec, 5000',
 '2, Mech, 12000',
 '4, Mech, 16000',
 '3, Elec, 18000',
 '9, CS, 16000',
 '5, Mech, 6000']

In [0]:
arrayRDD = feelistRDD.map(lambda x: x.split(","))

In [0]:
arrayRDD.collect()

Out[10]: [['1', ' Mech', ' 10000'],
 ['6', ' CS', ' 20000'],
 ['7', ' Elec', ' 5000'],
 ['2', ' Mech', ' 12000'],
 ['4', ' Mech', ' 16000'],
 ['3', ' Elec', ' 18000'],
 ['9', ' CS', ' 16000'],
 ['5', ' Mech', ' 6000'],
 ['1', ' Mech', ' 10000'],
 ['6', ' CS', ' 20000'],
 ['7', ' Elec', ' 5000'],
 ['2', ' Mech', ' 12000'],
 ['4', ' Mech', ' 16000'],
 ['3', ' Elec', ' 18000'],
 ['9', ' CS', ' 16000'],
 ['5', ' Mech', ' 6000'],
 ['1', ' Mech', ' 10000'],
 ['6', ' CS', ' 20000'],
 ['7', ' Elec', ' 5000'],
 ['2', ' Mech', ' 12000'],
 ['4', ' Mech', ' 16000'],
 ['3', ' Elec', ' 18000'],
 ['9', ' CS', ' 16000'],
 ['5', ' Mech', ' 6000']]

In [0]:
kvRDD = arrayRDD.map(lambda x: (x[1], float(x[2])))

In [0]:
kvRDD.collect()

Out[12]: [(' Mech', 10000.0),
 (' CS', 20000.0),
 (' Elec', 5000.0),
 (' Mech', 12000.0),
 (' Mech', 16000.0),
 (' Elec', 18000.0),
 (' CS', 16000.0),
 (' Mech', 6000.0),
 (' Mech', 10000.0),
 (' CS', 20000.0),
 (' Elec', 5000.0),
 (' Mech', 12000.0),
 (' Mech', 16000.0),
 (' Elec', 18000.0),
 (' CS', 16000.0),
 (' Mech', 6000.0),
 (' Mech', 10000.0),
 (' CS', 20000.0),
 (' Elec', 5000.0),
 (' Mech', 12000.0),
 (' Mech', 16000.0),
 (' Elec', 18000.0),
 (' CS', 16000.0),
 (' Mech', 6000.0)]

In [0]:
sumRDD = kvRDD.reduceByKey(lambda x,y: x+y)

### Action

In [0]:
sumRDD.collect()

Out[14]: [(' Elec', 69000.0), (' CS', 108000.0), (' Mech', 132000.0)]

### Transformations (Lazy)

In [0]:
feelistRDD = spark.sparkContext.textFile("/FileStore/tables/feelist.txt")
arrayRDD = feelistRDD.map(lambda x: x.split(","))
kvRDD = arrayRDD.map(lambda x: (x[1], float(x[2])))
sumRDD = kvRDD.reduceByKey(lambda x,y: x+y)

### Actions (strict)

In [0]:
sumRDD.collect()

Out[5]: [(' CS', 108000.0), (' Mech', 132000.0), (' Elec', 69000.0)]

### Executors: run tasks. Task is a smallest unit of computation.

### Jobs -> Stages -> Tasks

Shuffle and sort occurs at the end of each stage.